In [1]:
import os
import math
import pandas as pd
import xgboost as xgb
import pyarrow as pa
import pyarrow.dataset as pds

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.xgboost as mlflow_xgb
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature

import ray
from ray import tune
from ray.air import RunConfig, ScalingConfig
from ray.data import read_parquet
from ray.air.integrations.mlflow import MLflowLoggerCallback
from ray.train.xgboost import XGBoostTrainer
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

try:
    from ray.tune.callback import Callback      # Ray >= 2.6
except ImportError:
    from ray.tune.callbacks import Callback     # Older Ray
from utils import cluster_scaler_client

2025-09-01 18:47:01,680	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-09-01 18:47:01,760	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-09-01 18:47:01,897	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Pre-requsites

Configure the following user environment variables

1. AWS_ROLE_ARN - This is the AWS role being assumed via IR
2. S3_BUCKET_NAME

In [2]:
# Download dataset and push to S3

# Load dataset
data = fetch_california_housing(as_frame=True)
df = data.frame.rename(columns={"MedHouseVal": "median_house_value"})

# Split
train, tmp = train_test_split(df, test_size=0.3, random_state=42)
val, test  = train_test_split(tmp, test_size=0.5, random_state=42)

# Save locally
train.to_parquet("/tmp/train.parquet", index=False)
val.to_parquet("/tmp/val.parquet", index=False)
test.to_parquet("/tmp/test.parquet", index=False)

# Push to S3
!aws s3 cp /tmp/train.parquet s3://${S3_BUCKET_NAME}/end-to-end/california/train/
!aws s3 cp /tmp/val.parquet   s3://${S3_BUCKET_NAME}/end-to-end/california/val/
!aws s3 cp /tmp/test.parquet  s3://${S3_BUCKET_NAME}/end-to-end/california/test/

upload: ../../tmp/train.parquet to s3://ddl-wadkars/end-to-end/california/train/train.parquet
upload: ../../tmp/val.parquet to s3://ddl-wadkars/end-to-end/california/val/val.parquet
upload: ../../tmp/test.parquet to s3://ddl-wadkars/end-to-end/california/test/test.parquet


In [3]:
def ensure_mlflow_experiment(experiment_name: str) -> int:
    """
    Ensure an MLflow experiment exists and set it as current.

    If an experiment with `experiment_name` does not exist, create it. In both cases,
    set the active experiment so subsequent runs attach correctly.

    Parameters
    ----------
    experiment_name : str
        The MLflow experiment name.

    Returns
    -------
    str
        The experiment ID.

    Raises
    ------
    RuntimeError
        If the experiment lookup/creation fails.

    Notes
    -----
    - The MLflow tracking URI and token are pre-configured in Domino
    """
    try:
        exp = mlflow.get_experiment_by_name(experiment_name)
        if exp is None:
            exp_id = mlflow.create_experiment(
                experiment_name
            )
        else:
            exp_id = exp.experiment_id
        mlflow.set_experiment(experiment_name)
        return exp_id
    except Exception as e:
        raise RuntimeError(f"Failed to ensure experiment {experiment_name}: {e}")

In [11]:


def _s3p(root: str, sub: str) -> str:
    """Safe join for S3/posix URIs."""
    return f"{root.rstrip('/')}/{sub.lstrip('/')}"


def read_parquet_to_pandas(uri: str, columns=None, limit: int | None = None) -> pd.DataFrame:
    """
    Robust Parquet→pandas loader that bypasses Ray Data.
    Works with local paths and s3:// (PyArrow uses AWS_* env vars / IRSA).
    """
    ds = pds.dataset(uri.rstrip("/"), format="parquet")
    if limit is None:
        return ds.to_table(columns=columns).to_pandas()

    # Respect limit across files/row groups
    scanner = pds.Scanner.from_dataset(ds, columns=columns)
    batches, rows = [], 0
    for b in scanner.to_batches():
        batches.append(b)
        rows += len(b)
        if rows >= limit:
            return pa.Table.from_batches(batches)[:limit].to_pandas()
    return pa.Table.from_batches(batches).to_pandas()


def main(experiment_name:str,data_dir: str,num_workers: int = 4, cpus_per_worker: int = 1,  DEV_FAST: bool = False):
    """
    Quick knobs:
      - num_workers * cpus_per_worker = CPUs per trial.
      - trainer_resources={"CPU":0} so the driver doesn't steal a core.
      - PACK placement to keep trials tight.
      - max_concurrent_trials caps parallel trials.
      - num_boost_round / early_stopping_rounds control trial length.
      - nthread = cpus_per_worker to avoid oversubscription.
    """

    exp_id = ensure_mlflow_experiment(experiment_name)
    
    # Storage: local for dev, S3/your env otherwise
    RUN_STORAGE = os.environ.get("RAY_AIR_STORAGE", f"{data_dir}/air/xgb")
    TUNER_STORAGE = "/tmp/air-dev" if DEV_FAST else RUN_STORAGE
    FINAL_STORAGE = "/mnt/data/ddl-end-to-end-demo/air/final_fit" if DEV_FAST else RUN_STORAGE

    # Sanity: workers see IRSA env?
    @ray.remote
    def _peek():
        import os
        return {
            "ROLE": bool(os.environ.get("AWS_ROLE_ARN")),
            "TOKEN_FILE": os.environ.get("AWS_WEB_IDENTITY_TOKEN_FILE"),
            "REGION": os.environ.get("AWS_REGION"),
        }
    print("Worker env peek:", ray.get(_peek.remote()))

    # MLflow (experiment + parent run)
    CLUSTER_TRACKING_URI = os.environ["CLUSTER_MLFLOW_TRACKING_URI"]
    
    
    client = MlflowClient()


    parent = client.create_run(
        experiment_id=exp_id,
        tags={"mlflow.runName": "xgb_parent", "role": "tune_parent"},
    )
    parent_run_id = parent.info.run_id
    print("Parent run id:", parent_run_id)

    # Data (Ray Datasets for training/val)
    train_ds = read_parquet(_s3p(data_dir, "train"), parallelism=num_workers)
    val_ds   = read_parquet(_s3p(data_dir, "val"),   parallelism=num_workers)
    test_ds  = read_parquet(_s3p(data_dir, "test"),  parallelism=num_workers)
    print("Schema:", train_ds.schema())

    # Label + features
    label_col = "median_house_value"
    feature_cols = [c for c in train_ds.schema().names if c != label_col]
    keep = feature_cols + [label_col]
    train_ds = train_ds.select_columns(keep)
    val_ds   = val_ds.select_columns(keep)

    # DEV: trim Ray Datasets used for training; eval will bypass Ray entirely
    if DEV_FAST:
        train_ds = train_ds.limit(5_000)
        val_ds   = val_ds.limit(2_000)

    # --- Build test DataFrame without Ray (avoids 'Global node is not initialized') ---
    test_uri = _s3p(data_dir, "test")
    test_pdf = read_parquet_to_pandas(
        test_uri, columns=keep, limit=2_000 if DEV_FAST else None
    )

    # Search space
    param_space = {
        "params": {
            "objective": "reg:squarederror",
            "tree_method": "hist",
            "eval_metric": "rmse",
            "eta": tune.loguniform(1e-3, 3e-1),
            "max_depth": tune.randint(4, 12),
            "min_child_weight": tune.loguniform(1e-2, 10),
            "subsample": tune.uniform(0.6, 1.0),
            "colsample_bytree": tune.uniform(0.6, 1.0),
            "lambda": tune.loguniform(1e-3, 10),
            "alpha": tune.loguniform(1e-3, 10),
        },
        "num_boost_round": 300,
        "early_stopping_rounds": 20,
    }

    # Dev shortcuts
    if DEV_FAST:
        param_space["num_boost_round"] = 20
        param_space["early_stopping_rounds"] = 5
        num_workers = 1
        cpus_per_worker = 1
        NUM_SAMPLES = 5
        MAX_CONCURRENT = 3
        SAVE_ARTIFACTS = True
    else:
        NUM_SAMPLES = 30
        MAX_CONCURRENT = 3
        SAVE_ARTIFACTS = True

    # Threads per worker
    param_space["params"]["nthread"] = cpus_per_worker
    print("Per-trial CPUs =", num_workers * cpus_per_worker)

    # Scaling / placement
    scaling = ScalingConfig(
        num_workers=num_workers,
        use_gpu=False,
        resources_per_worker={"CPU": cpus_per_worker},
        trainer_resources={"CPU": 0},
        placement_strategy="PACK",
    )

    # Trainable
    trainer = XGBoostTrainer(
        label_column=label_col,
        params=param_space["params"],
        datasets={"train": train_ds, "valid": val_ds},
        num_boost_round=param_space["num_boost_round"],
        scaling_config=scaling,
    )

    # Search + scheduler
    MAX_T = int(param_space["num_boost_round"])
    GRACE = int(min(param_space.get("early_stopping_rounds", 1), MAX_T))
    algo = HyperOptSearch(metric="valid-rmse", mode="min")
    scheduler = ASHAScheduler(max_t=MAX_T, grace_period=GRACE, reduction_factor=3)

    # MLflow callback (child runs)
    mlflow_cb = MLflowLoggerCallback(
        tracking_uri=CLUSTER_TRACKING_URI,
        experiment_name=experiment_name,
        save_artifact=SAVE_ARTIFACTS,
        tags={"mlflow.parentRunId": parent_run_id},
    )

    # Tuner
    tuner = tune.Tuner(
        trainer.as_trainable(),
        run_config=RunConfig(
            name="xgb_from_s3_irsa",
            storage_path=TUNER_STORAGE,
            callbacks=[mlflow_cb],
        ),
        tune_config=tune.TuneConfig(
            search_alg=algo,
            scheduler=scheduler,
            metric="valid-rmse",
            mode="min",
            num_samples=NUM_SAMPLES,
            max_concurrent_trials=MAX_CONCURRENT,
        ),
        param_space={"params": param_space["params"]},
    )

    # Tune
    results = tuner.fit()
    best = results.get_best_result(metric="valid-rmse", mode="min")
    print("Best config:", best.config)
    print("Best valid RMSE:", best.metrics.get("valid-rmse"))

    # Final fit (train + val)
    merged = train_ds.union(val_ds)
    final_trainer = XGBoostTrainer(
        label_column=label_col,
        params=best.config["params"],
        datasets={"train": merged},
        num_boost_round=param_space["num_boost_round"],
        scaling_config=scaling,
        run_config=RunConfig(name="final_fit", storage_path=FINAL_STORAGE),
    )
    final_result = final_trainer.fit()
    final_ckpt = final_result.checkpoint

    # Load Booster from checkpoint
    with final_ckpt.as_directory() as ckpt_dir:
        print("Checkpoint dir:", ckpt_dir, "files:", os.listdir(ckpt_dir))
        candidates = ["model.json", "model.ubj", "model.xgb", "xgboost_model.json", "model"]
        model_path = next(
            (os.path.join(ckpt_dir, f) for f in candidates if os.path.exists(os.path.join(ckpt_dir, f))),
            None,
        )
        if not model_path:
            raise FileNotFoundError(f"No XGBoost model file found in checkpoint dir: {ckpt_dir}")
        booster = xgb.Booster()
        booster.load_model(model_path)

    # Driver-side eval (no Ray dependency)
    X_test = test_pdf.drop(columns=[label_col])
    
    dmat = xgb.DMatrix(X_test)
    y_pred = booster.predict(dmat)
    rmse = math.sqrt(((test_pdf[label_col].to_numpy() - y_pred) ** 2).mean())
    print(f"Test RMSE: {rmse:.4f}")

    
    # Log final under parent

    with mlflow.start_run(run_id=parent_run_id):
        X_example = X_test.head(5).copy()  
        y_example = booster.predict(xgb.DMatrix(X_example))
        sig = infer_signature(X_example, y_example)
        with mlflow.start_run(run_name="final_fit", nested=True):
            mlflow.log_params(best.config.get("params", {}))
            mlflow.log_dict({"label": label_col, "features": feature_cols}, "features.json")
            mlflow.log_metric("valid_rmse_best", float(best.metrics.get("valid-rmse")))
            mlflow.log_metric("test_rmse", float(rmse))
            mlflow_xgb.log_model(booster, artifact_path="model",signature=sig,input_example=X_example)
    run = client.get_run(parent_run_id)
    if run.info.status == "RUNNING":
        client.set_terminated(parent_run_id, "FINISHED")

In [5]:
from typing import Dict
def ensure_ray_connected(ray_envs: Dict[str,str], ray_ns:str):
    if ray.is_initialized():
        return
    # Reconnect to the running cluster (prefers ray:// if present)
    addr = None
    if "RAY_HEAD_SERVICE_HOST" in os.environ and "RAY_HEAD_SERVICE_PORT" in os.environ:
        addr = f"ray://{os.environ['RAY_HEAD_SERVICE_HOST']}:{os.environ['RAY_HEAD_SERVICE_PORT']}"
    ray.init(
        address=addr or "auto",
        runtime_env={"env_vars": ray_envs},   # same env you used earlier
        namespace=ray_ns,
        ignore_reinit_error=True,
    )

In [6]:
## Read Conf from Hydra
from hydra import compose, initialize
from omegaconf import OmegaConf
# Point Hydra to your conf/ directory
with initialize(config_path="conf"):
    cfg = compose(config_name="config", overrides=["env=dev"])
    #print(f"Running in {cfg.env} environment")
    #print(OmegaConf.to_yaml(cfg, resolve=True))
    
    app_name = cfg.app.name
    data_dir = cfg.app.data_dir
    experiment_name = cfg.mlflow.experiment_name    
    ray_workers = cfg.env.ray.num_workers
    cpus_per_worker = cfg.env.ray.cpus_per_worker
    dev_fast = cfg.env.ray.dev_fast
    #print(ray_workers)
    #print(dev_fast)
    


/tmp/ipykernel_3804/372615125.py:5: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="conf"):


In [7]:
# Disable tensorboard integration
os.environ["TUNE_DISABLE_AUTO_CALLBACK_LOGGERS"] = "1"

In [8]:
cluster_scaler_client.scale_cluster(cluster_kind="rayclusters",replicas=4)
cluster_scaler_client.wait_until_scaling_complete(cluster_kind="rayclusters")

http://ddl-cluster-scaler-svc.domino-field.svc.cluster.local/ddl_cluster_scaler/scale/rayclusters/ray-68b5e1f3b7910b03aaf26bb0
Status code 200
ray-68b5e1f3b7910b03aaf26bb0
http://ddl-cluster-scaler-svc.domino-field.svc.cluster.local/ddl_cluster_scaler/get/rayclusters/ray-68b5e1f3b7910b03aaf26bb0
Status code 200
Expected worker nodes 4
Current worker nodes ['ray-68b5e1f3b7910b03aaf26bb0-ray-worker-0']
Scaling not yet done...
ray-68b5e1f3b7910b03aaf26bb0
http://ddl-cluster-scaler-svc.domino-field.svc.cluster.local/ddl_cluster_scaler/get/rayclusters/ray-68b5e1f3b7910b03aaf26bb0
Status code 200
Expected worker nodes 4
Current worker nodes ['ray-68b5e1f3b7910b03aaf26bb0-ray-worker-0', 'ray-68b5e1f3b7910b03aaf26bb0-ray-worker-1', 'ray-68b5e1f3b7910b03aaf26bb0-ray-worker-2', 'ray-68b5e1f3b7910b03aaf26bb0-ray-worker-3']


True

In [ ]:
RAY_JOB_ENV = {
    "AWS_ROLE_ARN": os.environ.get("AWS_ROLE_ARN", ""),
    "AWS_WEB_IDENTITY_TOKEN_FILE": os.environ.get("AWS_WEB_IDENTITY_TOKEN_FILE", ""),
    "AWS_REGION": os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION", "us-east-1")),
    "AWS_DEFAULT_REGION": os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION", "us-east-1")),
    "TUNE_DISABLE_AUTO_CALLBACK_LOGGERS":"1",
    "TUNE_RESULT_BUFFER_LENGTH": "16",
    "TUNE_RESULT_BUFFER_FLUSH_INTERVAL_S": "3",    
    
}
ray.shutdown()
ensure_ray_connected(RAY_JOB_ENV,ray_ns=app_name)

main(experiment_name=experiment_name,data_dir=data_dir, num_workers=4, cpus_per_worker=1,DEV_FAST=dev_fast)

(TunerInternal pid=1377) AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.


(TunerInternal pid=1377) ╭──────────────────────────────────────────────────────────╮
(TunerInternal pid=1377) │ Configuration for experiment     xgb_from_s3_irsa        │
(TunerInternal pid=1377) ├──────────────────────────────────────────────────────────┤
(TunerInternal pid=1377) │ Search algorithm                 SearchGenerator         │
(TunerInternal pid=1377) │ Scheduler                        AsyncHyperBandScheduler │
(TunerInternal pid=1377) │ Number of trials                 30                      │
(TunerInternal pid=1377) ╰──────────────────────────────────────────────────────────╯
(TunerInternal pid=1377) 
(TunerInternal pid=1377) View detailed results here: ddl-wadkars/end-to-end/california/air/xgb/xgb_from_s3_irsa
(TunerInternal pid=1377) 
(TunerInternal pid=1377) Trial status: 1 PENDING
(TunerInternal pid=1377) Current time: 2025-09-01 11:49:12. Total running time: 0s
(TunerInternal pid=1377) Logical resource usage: 0/16 CPUs, 0/0 GPUs
(TunerInternal pid=1377) ╭───────

(XGBoostTrainer pid=297, ip=100.64.47.109) Started distributed worker processes: 
(XGBoostTrainer pid=297, ip=100.64.47.109) - (node_id=020c605741033b9152b4c9a349b6d58f62114a723ef561972a9adde1, ip=100.64.47.109, pid=343) world_rank=0, local_rank=0, node_rank=0
(XGBoostTrainer pid=297, ip=100.64.47.109) - (node_id=020c605741033b9152b4c9a349b6d58f62114a723ef561972a9adde1, ip=100.64.47.109, pid=344) world_rank=1, local_rank=1, node_rank=0
(XGBoostTrainer pid=297, ip=100.64.47.109) - (node_id=020c605741033b9152b4c9a349b6d58f62114a723ef561972a9adde1, ip=100.64.47.109, pid=345) world_rank=2, local_rank=2, node_rank=0
(XGBoostTrainer pid=297, ip=100.64.47.109) - (node_id=020c605741033b9152b4c9a349b6d58f62114a723ef561972a9adde1, ip=100.64.47.109, pid=346) world_rank=3, local_rank=3, node_rank=0
(RayTrainWorker pid=343, ip=100.64.47.109) [11:49:18] Task [xgboost.ray-rank=00000000]:361ba55c5007865d7aa0102a04000000 got rank 0
(RayTrainWorker pid=346, ip=100.64.47.109) [11:49:18] Task [xgboost.ray

(TunerInternal pid=1377) 
(TunerInternal pid=1377) Trial XGBoostTrainer_5f9c4ad7 started with configuration:
(TunerInternal pid=1377) ╭───────────────────────────────────────────────────────────────╮
(TunerInternal pid=1377) │ Trial XGBoostTrainer_5f9c4ad7 config                          │
(TunerInternal pid=1377) ├───────────────────────────────────────────────────────────────┤
(TunerInternal pid=1377) │ params/alpha                              0.39913158717439107 │
(TunerInternal pid=1377) │ params/colsample_bytree                    0.7816500895048499 │
(TunerInternal pid=1377) │ params/eta                               0.011894835892141157 │
(TunerInternal pid=1377) │ params/eval_metric                                       rmse │
(TunerInternal pid=1377) │ params/lambda                             0.08493766858268043 │
(TunerInternal pid=1377) │ params/max_depth                                            6 │
(TunerInternal pid=1377) │ params/min_child_weight                   0.0

(SplitCoordinator pid=548, ip=100.64.47.109) Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-09-01_11-12-11_235542_1/logs/ray-data
(SplitCoordinator pid=548, ip=100.64.47.109) Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> TaskPoolMapOperator[MapBatches(select_columns)] -> OutputSplitter[split(4, equal=True)]
(XGBoostTrainer pid=306, ip=100.64.25.122) Started distributed worker processes: 
(XGBoostTrainer pid=306, ip=100.64.25.122) - (node_id=aa7a1be9217ecfac6d99669d9eee6a9281027e3894db76bdace3f284, ip=100.64.25.122, pid=353) world_rank=0, local_rank=0, node_rank=0
(XGBoostTrainer pid=306, ip=100.64.25.122) - (node_id=aa7a1be9217ecfac6d99669d9eee6a9281027e3894db76bdace3f284, ip=100.64.25.122, pid=354) world_rank=1, local_rank=1, node_rank=0
(XGBoostTrainer pid=306, ip=100.64.25.122) - (node_id=aa7a1be9217ecfac6d99669d9eee6a9281027e3894db76bdace3f284, ip=100.64.25.122, pid=355) world_rank=2, local_rank=2, node_rank=0
(XGBoo

...

(TunerInternal pid=1377) 🏃 View run XGBoostTrainer_19391f24 at: http://ray-68b5e1f3b7910b03aaf26bb0-ray-proxy:8765/#/experiments/265/runs/2750eac9e2a24c4d819fdad2b8a86ff2
(TunerInternal pid=1377) 🧪 View experiment at: http://ray-68b5e1f3b7910b03aaf26bb0-ray-proxy:8765/#/experiments/265
(TunerInternal pid=1377) 
(TunerInternal pid=1377) Trial XGBoostTrainer_19391f24 completed after 180 iterations at 2025-09-01 13:16:53. Total running time: 1hr 27min 41s
(TunerInternal pid=1377) ╭──────────────────────────────────────────────────╮
(TunerInternal pid=1377) │ Trial XGBoostTrainer_19391f24 result             │
(TunerInternal pid=1377) ├──────────────────────────────────────────────────┤
(TunerInternal pid=1377) │ checkpoint_dir_name                              │
(TunerInternal pid=1377) │ time_this_iter_s                         0.00283 │
(TunerInternal pid=1377) │ time_total_s                             5.96184 │
(TunerInternal pid=1377) │ training_iteration                           180

(TunerInternal pid=1377) The `callbacks.on_trial_result` operation took 1.739 s, which may be a performance bottleneck.
(TunerInternal pid=1377) The `process_trial_result` operation took 1.740 s, which may be a performance bottleneck.
(TunerInternal pid=1377) Processing trial results took 1.740 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
(TunerInternal pid=1377) The `process_trial_result` operation took 1.740 s, which may be a performance bottleneck.
(XGBoostTrainer pid=4960, ip=100.64.8.234) [13:16:55] [16]	train-rmse:0.76297	valid-rmse:0.79986
(XGBoostTrainer pid=4960, ip=100.64.8.234) [13:16:56] [17]	train-rmse:0.75095	valid-rmse:0.79071
(TunerInternal pid=1377) The `callbacks.on_trial_result` operation took 1.756 s, which may be a performance bottleneck.
(TunerInternal pid=1377) The `process_trial_result` operation took 1.757 s, which may be a performance bottleneck.
(TunerInternal pid=1377) Processing trial results took 

(TunerInternal pid=1377) 
(TunerInternal pid=1377) Trial status: 28 TERMINATED | 2 RUNNING
(TunerInternal pid=1377) Current time: 2025-09-01 13:17:00. Total running time: 1hr 27min 48s
(TunerInternal pid=1377) Logical resource usage: 8.0/16 CPUs, 0/0 GPUs
(TunerInternal pid=1377) Current best trial: ca919a62 with valid-rmse=0.44804730023671846 and params={'params': {'objective': 'reg:squarederror', 'tree_method': 'hist', 'eval_metric': 'rmse', 'eta': 0.04092727091511802, 'max_depth': 10, 'min_child_weight': 2.6961269193856783, 'subsample': 0.7423996250457794, 'colsample_bytree': 0.7011982798079494, 'lambda': 0.004974847839491, 'alpha': 1.7026072301402462, 'nthread': 1}}
(TunerInternal pid=1377) ╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
(TunerInternal pid=1377) │ Trial name                st

(TunerInternal pid=1377) The `callbacks.on_trial_result` operation took 1.779 s, which may be a performance bottleneck.
(TunerInternal pid=1377) The `process_trial_result` operation took 1.780 s, which may be a performance bottleneck.
(TunerInternal pid=1377) Processing trial results took 1.780 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
(TunerInternal pid=1377) The `process_trial_result` operation took 1.780 s, which may be a performance bottleneck.


(TunerInternal pid=1377) 🏃 View run XGBoostTrainer_6d62774a at: http://ray-68b5e1f3b7910b03aaf26bb0-ray-proxy:8765/#/experiments/265/runs/932218ff0f6043db892cda8736c667be
(TunerInternal pid=1377) 🧪 View experiment at: http://ray-68b5e1f3b7910b03aaf26bb0-ray-proxy:8765/#/experiments/265
(TunerInternal pid=1377) 
(TunerInternal pid=1377) Trial XGBoostTrainer_6d62774a completed after 20 iterations at 2025-09-01 13:17:02. Total running time: 1hr 27min 50s
(TunerInternal pid=1377) ╭──────────────────────────────────────────────────╮
(TunerInternal pid=1377) │ Trial XGBoostTrainer_6d62774a result             │
(TunerInternal pid=1377) ├──────────────────────────────────────────────────┤
(TunerInternal pid=1377) │ checkpoint_dir_name                              │
(TunerInternal pid=1377) │ time_this_iter_s                         0.00281 │
(TunerInternal pid=1377) │ time_total_s                             5.30113 │
(TunerInternal pid=1377) │ training_iteration                            20 

In [ ]:
cluster_scaler_client.scale_cluster(cluster_kind="rayclusters",replicas=1)
cluster_scaler_client.wait_until_scaling_complete(cluster_kind="rayclusters")

In [ ]:
import ray, sys, pyarrow as pa, pandas as pd
print("DRIVER:", sys.version)
print("DRIVER pyarrow:", pa.__version__)
print("DRIVER pandas :", pd.__version__)

@ray.remote
def _env_probe():
    import sys, pyarrow as pa, pandas as pd
    return {
        "python": sys.version.split()[0],
        "pyarrow": pa.__version__,
        "pandas": pd.__version__,
    }

print("WORKER:", ray.get(_env_probe.remote()))
